In [14]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# --- 1. Carga de datos y filtrado base ---
prices = pd.read_excel('prices/Prices.xlsx', index_col=0)
dict_activos = pd.read_excel('dict/dict_temp.xlsx')
alloc = pd.read_excel('allocation/Allocation.xlsx')

# Solo Equities
equities = dict_activos[dict_activos['Asset Class'] == 'Equities']
tickers_equities = equities['Ticker'].unique()
tickers_validos = [t for t in tickers_equities if t in prices.columns]

prices_eq = prices[tickers_validos].copy()
min_hist = 6 * 252
validos_hist = prices_eq.notna().sum(axis=0) >= min_hist
prices_eq = prices_eq.loc[:, validos_hist]
prices_eq = prices_eq.dropna(axis=0, how='any')

tickers_final = list(prices_eq.columns)
equities = equities[equities['Ticker'].isin(tickers_final)].reset_index(drop=True)
assert list(prices_eq.columns) == list(equities['Ticker']), "Tickers no sincronizados"

print("\n--- Activos finales por región (después de todos los filtros) ---")
print(equities.groupby('Geografia').size())

# --- NUEVO: Define fecha límite para optimización (solo usa historia hasta aquí) ---
fecha_limite_train = '2023-12-31'

prices_eq_train = prices_eq.loc[:fecha_limite_train]

# --- 2. Estadísticos SOLO con training ---
retornos_train = np.log(prices_eq_train / prices_eq_train.shift(1)).dropna()
mu_train = retornos_train.mean().values
Sigma_train = retornos_train.cov().values
S_train = retornos_train.values
n = len(tickers_final)

# --- 3. Penalización por exceso de activos por región ---
def limite_region(peso_region):
    # Ajusta los límites a tu criterio
    if peso_region > 0.30:
        return 13
    elif peso_region > 0.10:
        return 6
    else:
        return 3

def cardinalidad_penalizacion(w, tickers_final, equities, alloc_valid, peso_min=0.01):
    penal = 0.0
    for _, row in alloc_valid.iterrows():
        geo = row['Geografia']
        peso_geo = row['Peso_normalizado']
        max_activos = limite_region(peso_geo)
        tickers_region = equities[equities['Geografia'] == geo]['Ticker'].tolist()
        idxs_region = [tickers_final.index(t) for t in tickers_region if t in tickers_final]
        n_activos = np.sum(w[idxs_region] >= peso_min)
        penal += max(0, n_activos - max_activos)
    return penal

# --- 4. Función objetivo: Sharpe penalizado por CVaR y cardinalidad ---
def cvar_loss(w, S, alpha=0.05):
    portf_rets = S @ w
    var = np.percentile(portf_rets, 100 * alpha)
    cvar = var + (1 / (alpha * len(portf_rets))) * np.sum(np.maximum(var - portf_rets, 0))
    return cvar

def neg_sharpe_penalized_card(w, mu, Sigma, S, alloc_valid, tickers_final, equities,
                              lambda_cvar=0.2, lambda_card=3.0, alpha=0.05):
    ret = np.dot(mu, w)
    vol = np.sqrt(np.dot(w, np.dot(Sigma, w)))
    sharpe = ret / vol if vol > 0 else -1e6
    cvar = cvar_loss(w, S, alpha)
    penal = cardinalidad_penalizacion(w, tickers_final, equities, alloc_valid, peso_min=0.01)
    return - (sharpe - lambda_cvar * cvar) + lambda_card * penal

# --- 5. Restricciones ---

def restriccion_region(idx, p):
    def fun(w):
        return np.sum(w[idx]) - p
    return fun

bounds = [(0, 1)] * n
constraints = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
]

# --- Allocation solo en regiones válidas y normalizadas ---
regiones_validas = []
for _, row in alloc.iterrows():
    geo = row['Geografia']
    activos_geo = equities[equities['Geografia'] == geo]['Ticker']
    if len(activos_geo) > 0:
        regiones_validas.append(geo)

alloc_valid = alloc[alloc['Geografia'].isin(regiones_validas)].copy()
total = alloc_valid['Peso'].sum()
alloc_valid['Peso_normalizado'] = alloc_valid['Peso'] / total

print("\n--- Allocation normalizada solo en regiones presentes ---")
print(alloc_valid[['Geografia', 'Peso', 'Peso_normalizado']])

for _, row in alloc_valid.iterrows():
    geo = row['Geografia']
    peso_obj = row['Peso_normalizado']
    activos_geo = equities[equities['Geografia'] == geo]['Ticker'].tolist()
    indices = [tickers_final.index(tkr) for tkr in activos_geo if tkr in tickers_final]
    if indices:
        constraints.append({'type': 'eq', 'fun': restriccion_region(indices, peso_obj)})

# --- 6. Optimización ---
w0 = np.ones(n) / n
res = minimize(
    neg_sharpe_penalized_card, w0,
    args=(mu_train, Sigma_train, S_train, alloc_valid, tickers_final, equities, 0.2, 3.0, 0.05),
    bounds=bounds, constraints=constraints,
    options={'maxiter': 1000}
)

# --- 7. Resultados ---
pesos_opt = res.x
resultado = pd.DataFrame({
    'Ticker': tickers_final,
    'Peso óptimo': pesos_opt
})
print("\n--- Pesos óptimos por activo (usando solo datos hasta", fecha_limite_train, ") ---")
print(resultado)

# --- 8. Chequeo: Suma por región y comparación con Allocation.xlsx ---
df_merge = pd.merge(resultado, equities[['Ticker', 'Geografia']], on='Ticker', how='left')
pesos_por_region = df_merge.groupby('Geografia')['Peso óptimo'].sum().reset_index()

comparacion = pd.merge(
    pesos_por_region,
    alloc[['Geografia', 'Peso']].rename(columns={'Peso': 'Peso_Restriccion'}),
    on='Geografia',
    how='outer'
)

print("\n--- Suma de pesos por región y comparación con restricciones ---")
print(comparacion)



--- Activos finales por región (después de todos los filtros) ---
Geografia
Asia ex Japón    37
China            21
Europe           20
India             1
Japan             3
Latam            28
USA              50
dtype: int64

--- Allocation normalizada solo en regiones presentes ---
       Geografia      Peso  Peso_normalizado
0            USA  0.700000          0.700000
1         Europe  0.125000          0.125000
2          Japan  0.071000          0.071000
3  Asia ex Japón  0.017382          0.017382
4          India  0.031465          0.031465
5          China  0.030000          0.030000
6          Latam  0.025152          0.025152


/var/folders/s9/mpd6xx3x1173hcqlz6vh5s_80000gn/T/ipykernel_9790/2323878229.py:64: RuntimeWarning: divide by zero encountered in matmul
  portf_rets = S @ w
/var/folders/s9/mpd6xx3x1173hcqlz6vh5s_80000gn/T/ipykernel_9790/2323878229.py:64: RuntimeWarning: overflow encountered in matmul
  portf_rets = S @ w
/var/folders/s9/mpd6xx3x1173hcqlz6vh5s_80000gn/T/ipykernel_9790/2323878229.py:64: RuntimeWarning: invalid value encountered in matmul
  portf_rets = S @ w



--- Pesos óptimos por activo (usando solo datos hasta 2023-12-31 ) ---
                Ticker   Peso óptimo
0       0JKT LN EQUITY  8.490032e-17
1    AACCHIA CI EQUITY  2.115412e-02
2       AAXJ US EQUITY  8.481641e-17
3    ABCAI2A LX EQUITY  0.000000e+00
4    AEEVI2E LX EQUITY  5.760004e-03
..                 ...           ...
156      XLU US EQUITY  1.988458e-23
157     XUCM LN EQUITY  5.676380e-17
158     XUFN LN EQUITY  0.000000e+00
159     XUIN LN EQUITY  8.231693e-02
160     XUTC LN EQUITY  5.527393e-02

[161 rows x 2 columns]

--- Suma de pesos por región y comparación con restricciones ---
       Geografia  Peso óptimo  Peso_Restriccion
0  Asia ex Japón     0.017382          0.017382
1          China     0.030000          0.030000
2         Europe     0.125000          0.125000
3          India     0.031465          0.031465
4          Japan     0.071000          0.071000
5          Latam     0.025152          0.025152
6            USA     0.700000          0.700000


In [15]:
import pandas as pd

# Usa el resultado y el merge con geografía que ya tienes:
df_merge = pd.merge(resultado, equities[['Ticker', 'Geografia']], on='Ticker', how='left')

# Solo activos con peso mayor a un umbral (por ejemplo, 1e-6 para limpiar los ceros "de máquina")
umbral = 1e-6
df_merge_filtrado = df_merge[df_merge['Peso óptimo'] > umbral]

# Ordena por región y peso descendente
df_merge_filtrado = df_merge_filtrado.sort_values(['Geografia', 'Peso óptimo'], ascending=[True, False])

# Ahora exporta: una hoja por región, cada hoja con Ticker y Peso óptimo
with pd.ExcelWriter('Seleccion_Activos_por_Region_activos_limitados.xlsx') as writer:
    for region in df_merge_filtrado['Geografia'].unique():
        df_region = df_merge_filtrado[df_merge_filtrado['Geografia'] == region][['Ticker', 'Peso óptimo']]
        df_region.to_excel(writer, sheet_name=region, index=False)


In [16]:
# Filtra activos seleccionados y normaliza los pesos para que sumen 1 (por si acaso)
umbral = 1e-6
portafolio_final = resultado[resultado['Peso óptimo'] > umbral].copy()
portafolio_final['Peso óptimo'] = portafolio_final['Peso óptimo'] / portafolio_final['Peso óptimo'].sum()
portafolio_final = portafolio_final.sort_values('Peso óptimo', ascending=False)

# Exporta a Excel
portafolio_final.rename(columns={'Peso óptimo': 'Peso'}).to_excel('Portafolio_limitado_activos.xlsx', index=False)
